In [1]:
import pandas as pd
import numpy as np
import math
import os
from test_class import Test
os.environ['PYTHONHASHSEED'] = str(50)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt

In [2]:
rounds = 50
dataset = pd.read_csv('RTT_data.csv')
data_args = {
    'data seed': 11,
    'distance clients': [0, 2, 3, 6],
    'distance augments': [0.5, 0.5, 0.5, 0.5],
    'tolerance': 5,
    'exclude dtypes': 'object',
    'drop labels': ['GroundTruthRange[m]'],
    'target labels': ['GroundTruthRange[m]'],
    'test size 1': 0.2,
    'test size 2': 0.25,
    'normalize': True,
    'client num': 10
}

Mt = (np.ones(rounds) * 5).astype('int32')
model_seed = 50
test = Test(dataset, data_args, Mt, model_seed)

In [3]:
test.split(scheme = 1)
test.display_metadata()

Number of samples: 29581
Features per sample: 10

Columns:
%Timestamp[s], GroundTruthPositionX[m], GroundTruthPositionY[m], GroundTruthPositionZ[m], ML_range_estimate[m], AP_positionX[m], AP_positionY[m], AP_positionZ[m], AP_index, ToD_factor[m]

Clients: 10
Total Client Training Samples: 23664 (80.00%)
Total Client Training Labels: 23664
Total Test Samples: 5917 (20.00%)
Total Test Labels: 5917


In [4]:
test.display_client_distribution()

Data Distribution
Client 1: 1.28%
Client 2: 0.67%
Client 3: 16.08%
Client 4: 23.99%
Client 5: 4.30%
Client 6: 2.22%
Client 7: 0.19%
Client 8: 23.75%
Client 9: 21.69%
Client 10: 5.82%

Distance Distribution w.r.t. Max Distance
Client 1: 68.42%
Client 2: 1.13%
Client 3: 84.83%
Client 4: 100.00%
Client 5: 24.36%
Client 6: 28.14%
Client 7: 58.71%
Client 8: 28.25%
Client 9: 54.60%
Client 10: 49.32%


In [5]:
# perform cross validation over different seeds
seeds = [1, 2, 3, 4, 5]
fedavg_scores = []
conf_scores = []
for s in seeds:
    test.SetDataSeed(s)
    cv_data = test.cross_validation(K = 7, score = 1, a = 0.1, l = 0.01, context = [0])
    s1 = cv_data['s1 score']
    fedavg_scores.append(s1[0])
    conf_scores.append(s1[1])

CV Pair 1
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 123.63s

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 156.81s

FedAvg Avg Test Error: 0.16682819445774144
ConFeddi Avg Test Error: 0.16412146579430031
FedAvg MSE: 0.07123302668333054
ConFeddi MSE: 0.06445027142763138

CV Pair 2
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 123.19s

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 151.96s

FedAvg Avg Test Error: 0.3016801973685505
ConFeddi Avg Test Error: 0.42834101756446574
FedAvg MSE: 0.140430748462677
ConFeddi MSE: 0.2082778513431549

CV Pair 3
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 122.27s

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 1

In [10]:
print(f'FedAvg Average S1 Score: {np.array(fedavg_scores).mean():.5f}')

FedAvg Average S1 Score: 0.27366


In [11]:
print(f'ConFeddi Average S1 Score: {np.array(conf_scores).mean():.5f}')

ConFeddi Average S1 Score: 0.25105


In [12]:
print(f'FedAvg S1 Score Variance: {np.array(fedavg_scores).var():.5f}')

FedAvg S1 Score Variance: 0.00613


In [13]:
print(f'ConFeddi S1 Score Variance: {np.array(conf_scores).var():.5f}')

ConFeddi S1 Score Variance: 0.00434
